<a href="https://colab.research.google.com/github/microprediction/microprediction/blob/master/notebook_examples_submission/enter_microprediction_bivariate_contests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install microprediction

(do restart the runtime when prompted)

# So, you're interested in copulas and multivariate distributions, eh? 

This is a colab notebook that makes it trivial to start participating in Microprediction z2-contests, which are a novel way of arriving at a bivariate distribution. You'll be submitting predictions to multiple [streams](https://www.microprediction.org/browse_streams.html) and who knows, you might even win the [daily prize](https://www.microprediction.com/competitions/daily). 

Another motivation: you might accidentally discover the best benchmarking platform for forecasting models, and use it to write your next paper. Help yourself to [data](https://microprediction.github.io/microprediction/data.html), by the way, and meet other time-series folks in the [slack](https://microprediction.github.io/microprediction/slack.html). We have an informal Google [meet](https://microprediction.github.io/microprediction/meet.html) too on Fridays at noon New York time where I'm happy to explain more.  

On the flip side, it's a little known fact that I can provide you free distributional prediction of anything you can measure. See the [instructions](https://microprediction.github.io/microprediction/publish.html) if you wan that. One day, half of all data science will be free or very close to it, but you know that already 'cause you read the [book](https://www.amazon.com/Microprediction-Building-Open-AI-Network/dp/0262047322), right?

Anyway, I've supplied this template that both initiate you into the microprediction community (by creating your secret identity) and then submits some predictions on your behalf. It should explain itself as it runs, sort of. Just click those arrows in sequence. 

### Step 0: Supply email ... if you want
Optional, but if you want to qualify for a cash prize, you need to add an email. 

In [1]:
EMAIL = "pcotton@intechinvestments.com"    # Edit
URL = 'https://github.com/microprediction/microprediction/blob/master/notebook_examples_submission/thanks_for_reaching_out.ipynb'                                    # If you want your code to be public (how spirited) then please supply a link to the code on Github or elsewhere
WRITE_KEY = None                              # Edit this after you run this colab notebook the first time, to update your submissions rather than creating a new WRITE_KEY

### Step 1: Create your secret identity, then submit
Just run this next cell and go get dinner. 
Before you run this cell again, update the WRITE_KEY above. 

In [ ]:
if WRITE_KEY is None:
  print('Creating a write key is one-off, but takes hours')
  from microprediction import new_key
  WRITE_KEY = new_key(difficulty=10)  # Better to use difficulty 12 actually

from microprediction import MicroWriter
mw = MicroWriter(write_key=WRITE_KEY)
ANIMAL = mw.animal_from_key(WRITE_KEY)
VERBOSE = False
if URL:
  mw.set_repository(url=URL) 

print({'private key':WRITE_KEY,'spirit animal ':ANIMAL,'url':URL})
print('Paste your private key in the dashboard at https://www.microprediction.org/')

import random
import time
import warnings
warnings.filterwarnings('ignore')
from copulas.multivariate import GaussianMultivariate
import pandas as pd
from microprediction import MicroWriter
from pprint import pprint

def fit_and_sample(lagged_zvalues,num:int, copula=None):
    """ Example of fitting a copula function, and sampling
           lagged_zvalues: [ [z1,z2,z3] ]  distributed N(0,1) margins, roughly
           copula : Something from https://pypi.org/project/copulas/
           returns: [ [z1, z2, z3] ]  representative sample
    """
    # A placeholder for generating quasi-monte carlo samples in 2-dim based on history of lagged values
    # Some more evenly spaced sampling would be preferable.
    # See https://www.microprediction.com/blog/intro for discussion

    df = pd.DataFrame(data=lagged_zvalues)
    if copula is None:
        copula = GaussianMultivariate()
    copula.fit(df)
    synthetic = copula.sample(num)
    return synthetic.values.tolist()


# List the rdps bivariate streams
NAMES = [ n for n in mw.get_stream_names() if 'z2~' in n and 'rdps' in n ]
for name in NAMES:
    delay = mw.DELAYS[-1]  # Only submit for the 1hr horizon
    lagged_zvalues = mw.get_lagged_zvalues(name=name, count= 5000)
    if len(lagged_zvalues)>20:
        zvalues = fit_and_sample(lagged_zvalues=lagged_zvalues, num=mw.num_predictions)
        try:
            res = mw.submit_zvalues(name=name, zvalues=zvalues, delay=delay )
            if VERBOSE:
                pprint(res)
        except Exception as e:
            print(e)
        time.sleep(1)  # Out of consideration for the system


Creating a write key is one-off, but takes hours


### What next?
You can: 


1.   Move this into a cron job and run it hourly or daily
2.   Modify the code that creates *values* from *lagged_values* so it is smarter. 
3.   Read up on [strategy](https://www.microprediction.com/blog/lottery) and [scoring](https://www.microprediction.com/blog/intro)
4.   Stop by some time and ask questions [here](https://microprediction.github.io/microprediction/meet.html)

